In [1]:
import pandas as pd
wallets_df = pd.read_excel(r"C:\Users\Dell\OneDrive\Desktop\Wallets.xlsx")
wallet_list = wallets_df['wallet_id'].dropna().unique().tolist()

print("Total Wallets:", len(wallet_list))
wallets_df.head()


Total Wallets: 103


wallet_id
0  0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1  0x06b51c6882b27cb05e712185531c1f74996dd988
2  0x0795732aacc448030ef374374eaae57d2965c16c
3  0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4  0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae

In [6]:
wallets_df.tail()

wallet_id
98   0xf60304b534f74977e159b2e159e135475c245526
99   0xf67e8e5805835465f7eba988259db882ab726800
100  0xf7aa5d0752cfcd41b0a5945867d619a80c405e52
101  0xf80a8b9cfff0febf49914c269fb8aead4a22f847
102  0xfe5a05c0f8b24fca15a7306f6a4ebb7dcf2186ac

In [7]:
wallets_df.size

103

In [8]:
wallets_df.shape

(103, 1)

In [9]:
wallets_df.describe()

wallet_id
count                                          103
unique                                         103
top     0x0039f22efb07a647557c7c5d17854cfd6d489ef3
freq                                             1

In [ ]:
wallets_df.head()

In [10]:
wallets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   wallet_id  103 non-null    object
dtypes: object(1)
memory usage: 956.0+ bytes


In [11]:
wallets_df.dtypes

wallet_id    object
dtype: object

In [12]:
wallets_df.isnull().sum()

wallet_id    0
dtype: int64

In [2]:
import requests
import time

def get_wallet_data(wallet_address):
    url = f"https://api.compound.finance/api/v2/account?address={wallet_address}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            account_data = response.json().get('accounts', [])
            if account_data:
                return {
                    'wallet_id': wallet_address,
                    'borrow': float(account_data[0].get('total_borrow_value_in_eth', 0)),
                    'supply': float(account_data[0].get('total_supply_value_in_eth', 0)),
                    'health': float(account_data[0].get('health', 0))
                }
        return {'wallet_id': wallet_address, 'borrow': 0, 'supply': 0, 'health': 0}
    except Exception as e:
        print(f"Error for {wallet_address}: {e}")
        return {'wallet_id': wallet_address, 'borrow': 0, 'supply': 0, 'health': 0}


In [3]:
wallet_data_list = []

for i, wallet in enumerate(wallet_list):
    print(f"Fetching {i+1}/{len(wallet_list)}: {wallet}")
    wallet_data_list.append(get_wallet_data(wallet))
    time.sleep(0.5)  # Pause to avoid API throttling

# Convert to DataFrame
df = pd.DataFrame(wallet_data_list)
df.head()


Fetching 1/103: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
Fetching 2/103: 0x06b51c6882b27cb05e712185531c1f74996dd988
Fetching 3/103: 0x0795732aacc448030ef374374eaae57d2965c16c
Fetching 4/103: 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
Fetching 5/103: 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
Fetching 6/103: 0x104ae61d8d487ad689969a17807ddc338b445416
Fetching 7/103: 0x111c7208a7e2af345d36b6d4aace8740d61a3078
Fetching 8/103: 0x124853fecb522c57d9bd5c21231058696ca6d596
Fetching 9/103: 0x13b1c8b0e696aff8b4fee742119b549b605f3cbc
Fetching 10/103: 0x1656f1886c5ab634ac19568cd571bc72f385fdf7
Fetching 11/103: 0x1724e16cb8d0e2aa4d08035bc6b5c56b680a3b22
Fetching 12/103: 0x19df3e87f73c4aaf4809295561465b993e102668
Fetching 13/103: 0x1ab2ccad4fc97c9968ea87d4435326715be32872
Fetching 14/103: 0x1c1b30ca93ef57452d53885d97a74f61daf2bf4f
Fetching 15/103: 0x1e43dacdcf863676a6bec8f7d6896d6252fac669
Fetching 16/103: 0x22d7510588d90ed5a87e0f838391aaafa707c34b
Fetching 17/103: 0x24b3460622d835c56d9a4fe352966b

wallet_id  borrow  supply  health
0  0x0039f22efb07a647557c7c5d17854cfd6d489ef3       0       0       0
1  0x06b51c6882b27cb05e712185531c1f74996dd988       0       0       0
2  0x0795732aacc448030ef374374eaae57d2965c16c       0       0       0
3  0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9       0       0       0
4  0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae       0       0       0

In [4]:
#Risk Scoring Formula
def calculate_score(row):
    borrow_weight = 0.4
    supply_weight = 0.3
    health_weight = 0.3

    borrow_score = min(row['borrow'], 1000) / 1000
    supply_score = min(row['supply'], 1000) / 1000
    health_score = min(row['health'], 10) / 10

    raw_score = (1 - borrow_score) * borrow_weight + supply_score * supply_weight + health_score * health_weight
    return int(raw_score * 1000)

df['score'] = df.apply(calculate_score, axis=1)
df[['wallet_id', 'borrow', 'supply', 'health', 'score']].head()


wallet_id  borrow  supply  health  score
0  0x0039f22efb07a647557c7c5d17854cfd6d489ef3       0       0       0    400
1  0x06b51c6882b27cb05e712185531c1f74996dd988       0       0       0    400
2  0x0795732aacc448030ef374374eaae57d2965c16c       0       0       0    400
3  0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9       0       0       0    400
4  0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae       0       0       0    400

In [5]:
final_df = df[['wallet_id', 'score']]
final_df.to_csv("wallet_risk_scores.csv", index=False)

print(" 'wallet_risk_scores.csv' saved successfully.")


 'wallet_risk_scores.csv' saved successfully.
